In [3]:
import pandas as pd

# read in the two files to be compared
path = "C:/Users/Eli/PycharmProjects/interview_skynet_software/interviewrequestskynetsoftware/"
actual_claims = pd.read_excel(path+"Actual_Claims.xlsx")
claims_mapping = pd.read_excel(path+"Caims_Mapping.xlsx")    # maps actual claims codes to industry claim codes
indusrty_events_cat = pd.read_excel(path+"Industry_Events.xlsx", sheet_name='CAT')
industry_events_ne = pd.read_excel(path+"Industry_Events.xlsx", sheet_name='NE')

In [4]:
actual_claims

,ClaimNo,Actual_Peril,Date_of_Loss,City,Province,Postal_Code,Loss_Amount
0,201600005175,L,2016-07-08,Grand Bend,ON,N0M 1T0,3634.89
1,201700005856,L,2017-04-04,Russell,ON,K4R 1E5,21100.65
2,201700005856,L,2017-04-04,Russell,ON,K4R 1E5,4217.75
3,201800006683,L,2018-01-12,Addison,ON,K0E 1A0,6723.60
4,201800006683,L,2018-01-12,Addison,ON,K0E 1A0,3287.69
...,...,...,...,...,...,...,...
5842,201400003763,M,2014-02-09,Nanaimo,BC,V9T 6E7,1361.88
5843,201400003804,M,2014-03-25,Nanaimo,BC,V9T 6E7,0.00
5844,201400003932,M,2014-07-04,Nanaimo,BC,V9T 6G1,0.00
5845,201500004645,M,2015-09-26,Mt. Washington,BC,V9N 8H9,0.00


In [5]:
claims_mapping

,Actual_Peril_Building,Actual_Peril_Contents,Industry_Peril
0,J,K,"['Peril_1' , 'Peril_2']"
1,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']"
2,N,O,['Peril_6']
3,P,Q,['Peril_7']
4,R,S,['Peril_8']
5,T,T,['Peril_1']
6,U,U,['Peril_7']


In [6]:
# add a column to actual claims containing the mapped to industry peril
# if value in actual peril column is in claims mapping, return industry peril vlue


peril_merge_1 = pd.merge(actual_claims, 
                       claims_mapping, 
                       how='inner', 
                       left_on=['Actual_Peril'],
                       right_on=['Actual_Peril_Building'])

peril_merge_2 = pd.merge(actual_claims, 
                       claims_mapping, 
                       how='inner', 
                       left_on=['Actual_Peril'],
                       right_on=['Actual_Peril_Contents'])

peril_merged = peril_merge_1.append(peril_merge_2, ignore_index=True)

peril_merged

,ClaimNo,Actual_Peril,Date_of_Loss,City,Province,Postal_Code,Loss_Amount,Actual_Peril_Building,Actual_Peril_Contents,Industry_Peril
0,201600005175,L,2016-07-08,Grand Bend,ON,N0M 1T0,3634.89,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']"
1,201700005856,L,2017-04-04,Russell,ON,K4R 1E5,21100.65,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']"
2,201700005856,L,2017-04-04,Russell,ON,K4R 1E5,4217.75,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']"
3,201800006683,L,2018-01-12,Addison,ON,K0E 1A0,6723.60,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']"
4,201800006683,L,2018-01-12,Addison,ON,K0E 1A0,3287.69,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']"
...,...,...,...,...,...,...,...,...,...,...
6744,201500004720,U,2015-10-31,Victoria,BC,V8Z 3V9,15471.54,U,U,['Peril_7']
6745,201600005338,U,2016-08-04,Victoria,BC,V8Z 3P9,11423.71,U,U,['Peril_7']
6746,201500004210,U,2015-02-25,Victoria,BC,V8V 3G6,2636.89,U,U,['Peril_7']
6747,201500004170,U,2015-01-18,Victoria,BC,V8V 3L1,3131.56,U,U,['Peril_7']


In [7]:
# test to make sure that we have not lost any information

'''
pd.testing.assert_frame_equal(actual_claims_check, 
                              peril_merge_check, 
                              check_like=True,    # ignore ordering
                              check_dtype=False)
'''

actual_claims_check = actual_claims.sort_values(by=['ClaimNo', 'Actual_Peril', 'Loss_Amount']).drop_duplicates()
actual_claims_check= actual_claims_check.reset_index(drop=True)

peril_merge_check = peril_merged.sort_values(by=['ClaimNo','Actual_Peril','Loss_Amount']).drop_duplicates()

peril_merge_check = peril_merge_check.loc[:, 'ClaimNo':'Loss_Amount']
peril_merge_check = peril_merge_check.reset_index(drop=True)

actual_claims_check.compare(peril_merge_check) # shows the rows that are different
actual_claims_check.equals(peril_merge_check) # checks equality

True

In [8]:
indusrty_events_cat

,ID,Name,Province,Locations,Start_Date,End_Date,Perils
0,2008-01-Cat-0000,Ontario Windstorm,ON,"Greater Toronto Area, Prince Edward County",2008-01-09,2008-01-09,['Cat_WindAndHail']
1,2008-06-Cat-0001,Montreal Severe Storm,QC,Montreal and Surrounding Regions,2008-06-10,2008-06-10,['Cat_WindAndHail']
2,2008-07-Cat-0002,Swift Current Hailstorm,SK,Swift Current,2008-07-09,2008-07-09,['Cat_WindAndHail']
3,2008-08-Cat-0003,Red Deer Hailstorm,AB,Red Deer,2008-08-08,2008-08-08,['Cat_WindAndHail']
4,2008-12-Cat-0004,Vancouver Winterstorm,BC,Vancouver,2008-12-21,2008-12-21,['Cat_Winterstorm']
...,...,...,...,...,...,...,...
109,2019-01-Cat-0109,Quebec & Maritimes Flooding and Wind,"QC, NB, NS, PE","Saint John, Sussex, Moncton, Montreal, Quebec",2019-01-24,2019-01-25,"['Cat_Flood', 'Cat_Winterstorm']"
110,2019-02-Cat-0110,Ontario and Quebec Windstorm and Flooding,"ON, QC","Niagara region, Hamilton, Greater Toronto Area",2019-02-24,2019-02-25,"['Cat_Flood', 'Cat_Winterstorm']"
111,2019-02-Cat-0111,Ontario and Quebec Early-February Thaw,"ON, QC","GTA, Ottawa, Orillia, Gatineau, Montreal, She...",2019-02-03,2019-02-05,"['Cat_Flood', 'Cat_Winterstorm']"
112,2019-03-Cat-0112,Southern Ontario & Quebec Snowmelt and Rain,"ON, QC",Greater Toronto Area,2019-03-09,2019-03-11,"['Cat_Flood', 'Cat_Winterstorm']"


In [16]:
industry_events_ne

,ID,Region,Provinces,Locations,Start_Date,End_Date,Industry_Peril
0,2008-04-NE-0000,Quebec Wind and Rain Storm,QC,St. Lawrence Valley,2008-04-01,2008-04-02,"Peril_9, Peril_4"
1,2008-04-NE-0010,Quebec Flooding,QC,Various Regions,2008-04-28,2008-05-02,Peril_9
2,2008-04-NE-0020,Saint John River Flooding,NB,"Saint John River, Edmunston, Fredericton, Maug...",2008-04-29,2008-05-03,Peril_9
3,2008-06-NE-0030,Camrose Hailstorm,AB,Camrose,2008-06-07,2008-06-07,Peril_4
4,2008-06-NE-0040,Greater Toronto Area Hailstorm,ON,GTA,2008-06-16,2008-06-16,Peril_4
...,...,...,...,...,...,...,...
93,2018-06-NE-0040,Alberta and Western Saskatchewan Severe Storms,"AB, SK","Edmonton, Medicine Hat, Lloydminster",2018-06-09,2018-06-12,"Peril_9, Peril_4"
94,2018-07-NE-0050,Prairie Severe Storms,"AB, SK, MB","Edmonton, Lloydminster, Pickardville",2018-07-20,2018-07-22,"Peril_9, Peril_4"
95,2018-07-NE-0060,Saskatchewan Storms,"AB, SK","Kindersley, Val Marie, Saskatoon",2018-07-10,2018-07-11,Peril_4
96,2018-11-NE-0080,Eastern Canada Wind & Rain Storm,"ON, QC, NB, NS, PE, NL","Moncton, Fredericton, Halifax, St. John's, Cor...",2018-11-01,2018-11-04,"Peril_9, Peril_4"


We want to see if the Actual Claims are associated with Industry Events, we also want to know if the claim affected the contents or the building.
To do this we will check:
  * if date_of_loss is contained in the start_date end_date interval
  * if the province matches
  * if the peril matches


In [17]:
import datetime

date_1 = datetime.datetime.strptime('2015-02-25', "%Y-%m-%d")

# look for events between Start_Date and End_Date with an error of 10 days
industry_events_ne.loc[(industry_events_ne.Start_Date<=date_1 + datetime.timedelta(days=10)) & 
                       (industry_events_ne.End_Date>=date_1 - datetime.timedelta(days=10))]

,ID,Region,Provinces,Locations,Start_Date,End_Date,Industry_Peril
61,2015-02-NE-0000,Atlantic Canada Snowstorm,"NB, NS, PE",Atlantic Canada,2015-02-15,2015-02-17,Peril_5


In [9]:
# First Full Outer Join Dataframes 
# (Requires a Common Column in Pandas Unlike SQL)
peril_merged['fake key'] = 1
industry_events_ne['fake key'] = 1
outer_join = pd.merge(peril_merged, industry_events_ne, how='outer', on='fake key')

# Now Filter Back down to Desired Rows/Columns i.e. those that are between the start and end of the cat event
desired_rows = outer_join.query('Date_of_Loss < End_Date and Date_of_Loss > Start_Date')
desired_rows

# match provinces
desired_rows.loc[desired_rows['Province'] == desired_rows['Provinces']].drop('fake key', axis=1).reset_index(drop=True)

# we should get a table containing the actual_peril, whether it affected the building, contents or both, and an industry event likely 
# associated to the claim

,ClaimNo,Actual_Peril,Date_of_Loss,City,Province,Postal_Code,Loss_Amount,Actual_Peril_Building,Actual_Peril_Contents,Industry_Peril_x,ID,Region,Provinces,Locations,Start_Date,End_Date,Industry_Peril_y
0,201600005022,L,2016-06-15,Dawson Creek,BC,V1G 2X2,46278.14,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']",2016-06-NE-0030,British Columbia Flooding,BC,"West Vancouver, Peace River Region, Dawson Cre...",2016-06-14,2016-06-16,Peril_9
1,201600005022,L,2016-06-15,Dawson Creek,BC,V1G 2X2,23865.17,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']",2016-06-NE-0030,British Columbia Flooding,BC,"West Vancouver, Peace River Region, Dawson Cre...",2016-06-14,2016-06-16,Peril_9
2,201500004328,N,2015-05-17,Landmark,MB,R0A 0X0,0.00,N,O,['Peril_6'],2015-05-NE-0010,Southern Manitoba Severe Storms,MB,"Southern Manitoba, Winnipeg",2015-05-16,2015-05-18,"Peril_9, Peril_5"
3,201500004319,N,2015-05-17,Winnipeg,MB,R3Y 1S4,16999.63,N,O,['Peril_6'],2015-05-NE-0010,Southern Manitoba Severe Storms,MB,"Southern Manitoba, Winnipeg",2015-05-16,2015-05-18,"Peril_9, Peril_5"
4,201500004327,N,2015-05-17,St. Germain South,MB,R5A 1H3,4561.73,N,O,['Peril_6'],2015-05-NE-0010,Southern Manitoba Severe Storms,MB,"Southern Manitoba, Winnipeg",2015-05-16,2015-05-18,"Peril_9, Peril_5"
5,201500004512,N,2015-08-16,Nakusp,BC,V0G 1R1,12966.05,N,O,['Peril_6'],2015-08-NE-0070,Rock Creek Wildfire,BC,Regional District of Kootenay Boundary,2015-08-13,2015-08-25,Peril_2
6,201500004512,N,2015-08-16,Nakusp,BC,V0G 1R1,796.00,N,O,['Peril_6'],2015-08-NE-0070,Rock Creek Wildfire,BC,Regional District of Kootenay Boundary,2015-08-13,2015-08-25,Peril_2
7,201500004506,N,2015-08-17,Fort St. John,BC,V1J 5X7,0.00,N,O,['Peril_6'],2015-08-NE-0070,Rock Creek Wildfire,BC,Regional District of Kootenay Boundary,2015-08-13,2015-08-25,Peril_2
8,201500004309,P,2015-05-17,New Bothwell,MB,R0A 1C0,12400.00,P,Q,['Peril_7'],2015-05-NE-0010,Southern Manitoba Severe Storms,MB,"Southern Manitoba, Winnipeg",2015-05-16,2015-05-18,"Peril_9, Peril_5"
9,201600005019,P,2016-06-15,Chetwynd,BC,V0C 1J0,54200.13,P,Q,['Peril_7'],2016-06-NE-0030,British Columbia Flooding,BC,"West Vancouver, Peace River Region, Dawson Cre...",2016-06-14,2016-06-16,Peril_9


In [10]:
# repeat the above process on cat dataframe

peril_merged['fake key'] = 1
indusrty_events_cat['fake key'] = 1
outer_join = pd.merge(peril_merged, indusrty_events_cat, how='outer', on='fake key')

# Now Filter Back down to Desired Rows/Columns
desired_rows_2 = outer_join.query('Date_of_Loss < End_Date and Date_of_Loss > Start_Date')
#desired_columns = ['Date_of_Loss', 'Industry_Peril', 'ID']
desired_rows_2

# match provinces
desired_rows_2.loc[desired_rows_2['Province_x'] == desired_rows_2['Province_y']].drop('fake key', axis=1).reset_index(drop=True)

,ClaimNo,Actual_Peril,Date_of_Loss,City,Province_x,Postal_Code,Loss_Amount,Actual_Peril_Building,Actual_Peril_Contents,Industry_Peril,ID,Name,Province_y,Locations,Start_Date,End_Date,Perils
0,200900002054,L,2009-02-12,Cottam,ON,N0R 1B0,25813.4,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']",2009-02-Cat-0006,Southwestern Ontario Flood,ON,Southwestern Ontario,2009-02-11,2009-02-13,['Cat_Flood']
1,200900002056,L,2009-02-12,Aylmer,ON,N5H 2R3,0.0,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']",2009-02-Cat-0006,Southwestern Ontario Flood,ON,Southwestern Ontario,2009-02-11,2009-02-13,['Cat_Flood']
2,201700006368,L,2017-07-14,Mackenzie,BC,V0J 2C0,0.0,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']",2017-07-Cat-0093,Elephant Hill Wildfire,BC,"Ashcroft, Cache Creek, Loon Lake, Pressy Lake",2017-07-07,2017-09-27,['Cat_Wildfire']
3,201700006203,L,2017-07-08,150 Mile House,BC,V0K 2G0,2000.0,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']",2017-07-Cat-0093,Elephant Hill Wildfire,BC,"Ashcroft, Cache Creek, Loon Lake, Pressy Lake",2017-07-07,2017-09-27,['Cat_Wildfire']
4,201700006204,L,2017-07-08,150 Mile House,BC,V0K 2G0,2000.0,L,M,"['Peril_3' , 'Peril_4', 'Peril_5']",2017-07-Cat-0093,Elephant Hill Wildfire,BC,"Ashcroft, Cache Creek, Loon Lake, Pressy Lake",2017-07-07,2017-09-27,['Cat_Wildfire']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,201700006065,U,2017-07-08,Williams Lake,BC,V2G 4P9,2000.0,U,U,['Peril_7'],2017-07-Cat-0093,Elephant Hill Wildfire,BC,"Ashcroft, Cache Creek, Loon Lake, Pressy Lake",2017-07-07,2017-09-27,['Cat_Wildfire']
371,201700006114,U,2017-07-08,Williams Lake,BC,V2G 4P9,2000.0,U,U,['Peril_7'],2017-07-Cat-0093,Elephant Hill Wildfire,BC,"Ashcroft, Cache Creek, Loon Lake, Pressy Lake",2017-07-07,2017-09-27,['Cat_Wildfire']
372,201700006149,U,2017-07-08,Williams Lake,BC,V2G 4S9,2000.0,U,U,['Peril_7'],2017-07-Cat-0093,Elephant Hill Wildfire,BC,"Ashcroft, Cache Creek, Loon Lake, Pressy Lake",2017-07-07,2017-09-27,['Cat_Wildfire']
373,201700006229,U,2017-07-08,Williams Lake,BC,V2G 4T2,2000.0,U,U,['Peril_7'],2017-07-Cat-0093,Elephant Hill Wildfire,BC,"Ashcroft, Cache Creek, Loon Lake, Pressy Lake",2017-07-07,2017-09-27,['Cat_Wildfire']
